In [1]:
from vanna.openai import OpenAI_Chat
from openai import AzureOpenAI
from vanna.vannadb import VannaDB_VectorStore
from dotenv import load_dotenv
import os

load_dotenv()

# from vanna.local import LocalContext_OpenAI
# vn = LocalContext_OpenAI()
vannaModel = os.getenv('MY_VANNA_MODEL')
vannaApiKey = os.getenv('MY_VANNA_API_KEY')
azureOpenAIApiKey = os.getenv('API_KEY')
azureOpenAIBaseUrl = os.getenv('BASE_URL')
azureOpenAIVersion = os.getenv('API_VERSION')


class MyVanna(VannaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        MY_VANNA_MODEL = vannaModel  # Vanna.ai model name
        MY_VANNA_API_KEY = vannaApiKey  # Vanna.ai API key
        VannaDB_VectorStore.__init__(self, vanna_model=MY_VANNA_MODEL, vanna_api_key=MY_VANNA_API_KEY, config=config)
        openai_client = AzureOpenAI(
            api_key=azureOpenAIApiKey,  # Azure OpenAI API key
            base_url=azureOpenAIBaseUrl,  # Azure OpenAI endpoint URL
            api_version=azureOpenAIVersion  # Azure OpenAI version
        )
        OpenAI_Chat.__init__(self, client=openai_client, config=config)


vn = MyVanna(config={vannaModel: 'gpt-4o-1'})
vn.connect_to_postgres(host='localhost', dbname='vanna_rag', user='berdn70s', password='123123', port='5432')
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn, allow_llm_to_see_data=True).flask_app

DependencyError: You need to install required dependencies to execute this method, run command: 
pip install vanna[postgres]

zsh:1: no matches found: vanna[openai,postgres,chromadb]
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Hem soruyu verip hem cevabi koymak, modeli egitmenin en direkt, efektif ve faydali yolu.
vn.train(
    question="What is the average sales of our customers?", 
    sql="SELECT AVG(sales) FROM customers"
)

'46e452793a2bdae69e2ea33ed16b8e8d-sql'

In [16]:
import os

directory_path = '/Users/berdn90s/vannaAI/ddl/queries'
ddl_path = '/Users/berdn90s/vannaAI/ddl/ddl.sql'

def read_sql(file_path):
    with open(file_path, 'r') as file:
        return file.read()

ddl_query = read_sql(ddl_path)

for filename in os.listdir(directory_path):
    if filename.endswith('.sql'):   
        file_path = os.path.join(directory_path, filename)
        sql_query = read_sql(file_path)
        print(f"Training on query from {filename}")
        vn.train(sql=sql_query)
        vn.train(ddl=ddl_query)  # Her sql querysi icin ddli tekrar train etmek?

print("Training completed for all SQL queries and DDL.")


Training on query from ip.sql
Using model gpt-4O for 101.0 tokens (approx)


NotFoundError: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}

In [2]:

from vanna.flask import VannaFlaskApp
vn.connect_to_postgres(host='localhost', dbname='vanna_rag', user='berdn70s', password='123123', port='5432')
app = VannaFlaskApp(vn,allow_llm_to_see_data=True)
app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on


In [15]:
vn.ask(question= 'What is the total number of unique network protocol?')

You have exceeded the free daily token limit. Please go to https://vanna.ai/account/profile to upgrade your subscription.
Couldn't run sql:  syntax error at or near "You"
LINE 1: You have exceeded the free daily token limit. Please go to h...
        ^



In [16]:
import pandas as pd

data = pd.read_csv('/Users/berdn90s/Downloads/weatherAUS.csv')
data.head()


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [2]:
import pandas as pd
import psycopg2

df = pd.read_csv('/Users/berdn90s/Downloads/weatherAUS.csv')

conn = psycopg2.connect(
    dbname="vanna_rag",  
    user="berdn70s",
    password="123123",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

for idx, row in df.iterrows():
    cur.execute(
        "INSERT INTO weather_logs (Date, Location, MinTemp, MaxTemp, Rainfall, Evaporation, Sunshine, WindGustDir, WindGustSpeed, WindDir9am, WindDir3pm, WindSpeed9am, WindSpeed3pm, Humidity9am, Humidity3pm, Pressure9am, Pressure3pm, Cloud9am, Cloud3pm, Temp9am, Temp3pm, RainToday) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
        (row['Date'], row['Location'], row['MinTemp'], row['MaxTemp'], row['Rainfall'], row['Evaporation'], row['Sunshine'], row['WindGustDir'], row['WindGustSpeed'], row['WindDir9am'], row['WindDir3pm'], row['WindSpeed9am'], row['WindSpeed3pm'], row['Humidity9am'], row['Humidity3pm'], row['Pressure9am'], row['Pressure3pm'], row['Cloud9am'], row['Cloud3pm'], row['Temp9am'], row['Temp3pm'], row['RainToday'])
    )
conn.commit()

cur.close()
conn.close()


NumericValueOutOfRange: integer out of range
